# Confirming gene models with isoform-level RNA-seq data

## Motivation

Our gene prediction with BRAKER3 produced a low number of gene models (~11.5k). We suspect this
might be because GeneMark-ET/StringTie are too aggressive with exons/genes that are close to each
other, merging them and messing up the gene models. We will use the isoform-level RNA-seq data to
confirm some of the gene models.

In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm

I extracted all transcript lines from the corresponding gff/gtf files and used bedtools to intersect
them:

```bash
$ ISOSEQ=/lisc/scratch/zoology/pycnogonum/transcriptome/isoseq/isoseq_confirmed.gff
$ BRAKER=/lisc/scratch/zoology/pycnogonum/genome/draft/braker/braker/braker_proposed.gtf
$ bedtools intersect -wao -b $ISOSEQ -a $BRAKER > brak_iso.bed
```

In [2]:
gtf = ["seqname", "source", "feature", "start", "end", "score", "strand", "frame", "attributes"]

# iso2bra = pd.read_csv("/Volumes/scratch/pycnogonum/genome/draft/annot_merge/iso_brak.bed", sep="\t", header=None)
# iso2bra.columns = ["iso_" + x for x in gtf] + ["bra_" + x for x in gtf] + ["overlap"]
# iso2bra["iso_gene"] = iso2bra["iso_attributes"].str.extract(r'gene_id "(.*?)";')
# iso2bra["bra_gene"] = iso2bra["bra_attributes"].str.split("\.").str[0]
# iso2bra["bra_gene"] = iso2bra["bra_gene"].replace("", np.nan)

bra2iso = pd.read_csv("/Volumes/scratch/pycnogonum/genome/draft/annot_merge/brak_iso.bed", sep="\t", header=None)
bra2iso.columns = ["bra_" + x for x in gtf] + ["iso_" + x for x in gtf] + ["overlap"]
bra2iso["bra_gene"] = bra2iso["bra_attributes"].str.split(".").str[0]
bra2iso["iso_gene"] = bra2iso["iso_attributes"].str.extract(r'gene_id "(.*?)";')

Now to find overlaps:

In [3]:
def contained(df, a_start="bra_start", a_end="bra_end", b_start="iso_start", b_end="iso_end"):
    """Checks if sequence A is completely contained within sequence B or vice versa.

    Parameters
    ----------
    df : pd.DataFrame
        A dataframe of a bedtools intersect file.
    a_start : str, optional
        the start position of sequence A, by default "bra_start"
    a_end : str, optional
        the end position of sequence B, by default "bra_end"
    b_start : str, optional
        The start position of sequence B, by default "iso_start"
    b_end : str, optional
        the end position of sequence B, by default "iso_end"

    Returns
    -------
    bool
        returns True if the sequence is contained, False otherwise
    """
    if df["bra_strand"] != df["iso_strand"]:
        return False # make sure that we are comparing genes on the same strand
    if df[b_start] < 0 or df[b_end] < 0 or df[a_start] < 0 or df[a_end] < 0:
        return False
    # first check if the braker gene is completely contained within the isoseq one
    if df[a_start] >= df[b_start] and df[a_end] <= df[b_end]:
        return True
    # then check if the braker gene completely contains the isoseq one
    if df[a_start] <= df[b_start] and df[a_end] >= df[b_end]:
        return True
    return False

In [4]:
genes = bra2iso.groupby("bra_gene")

In [5]:
def calc_overlap(df):
    bra_len = df.iloc[0]["bra_end"] - df.iloc[0]["bra_start"]
    # print(bra_len)
    seq = np.zeros(bra_len, dtype=bool)
    for i, transcript in df.iterrows():
        start = transcript["iso_start"] - transcript["bra_start"]
        start = np.max((0, start))
        end = transcript["iso_end"] - transcript["bra_start"]
        # print(end, len(seq))
        end = np.min([bra_len, end])
        seq[start:end] = True
    return np.sum(seq) / bra_len

In [6]:
braker_unique = []
significant = []
unknown = []
truncate = []


for gene in tqdm(genes.groups):
    try:
        transcripts = genes.get_group(gene)
    except KeyError:
        unknown.append(gene)
        continue
    # first see how many candidate ISO-seq genes this BRAKER gene overlaps with
    # if there is zero, then it means no possible corresponding ISO-seq gene was found
    no_iso = transcripts.groupby("iso_gene").size().shape[0] # how many ISO-seq transcripts match this BRAKER gene?
    if no_iso == 0: 
        braker_unique.append(gene)
        continue
    
    # if there is one it means we have an 1-to-1 correspondence
    if no_iso == 1:
        # try to verify that most BRAKER isoforms are contained within the ISO-seq isoforms
        # (or vice versa)
        no_contained = transcripts.apply(contained, axis=1).sum()
        if no_contained / transcripts.shape[0] >= 0.4:
            significant.append(gene)
            continue

        # test if the ISO-seq isoforms cumulatively cover at least 50% of the BRAKER gene
        total_overlap = transcripts.groupby("bra_attributes").apply(calc_overlap, include_groups=False)
        if np.mean(total_overlap) >= 0.5:
            significant.append(gene)
            continue

    # if we have arrived here, all else failed and this gene needs to go for manual curation
    unknown.append(gene)

100%|██████████| 11451/11451 [00:03<00:00, 2913.12it/s]


In [7]:
len(significant), len(braker_unique), len(unknown), np.sum([len(significant), len(braker_unique), len(unknown)])

(7591, 3596, 264, np.int64(11451))

At this point the strategy should be to copy the ISO-seq and "braker_unique" gene models from their
respective GTF files to a new, merged one. This should already take us from the 11,451 BRAKER gene
models to 8,904 + 3,442 = 12,346 gene models. We can then screen the unknown modules further.

In [8]:
isoseq_loc = "/Volumes/scratch/pycnogonum/transcriptome/isoseq/collapsed.gff"
braker_loc = "/Volumes/scratch/pycnogonum/genome/draft/braker/braker/braker.gtf"

In [17]:
def gene_string(gene_id, start, stop, last_line):
    chunks = last_line.split("\t")

    seqid = chunks[0]
    source = chunks[1]
    seq_type = "gene"
    score = "."
    strand = chunks[6]
    phase = "."
    attributes = f"ID={gene_id};gene_id={gene_id};"
    string = f"{seqid}\t{source}\t{seq_type}\t{start}\t{stop}\t{score}\t{strand}\t{phase}\t{attributes};\n"
    return string

def transcript_string(gene_id, transcript_id, line):
    chunks = line.split("\t")

    seqid = chunks[0]
    source = chunks[1]
    seq_type = "mRNA"
    start = chunks[3]
    stop = chunks[4]
    score = chunks[5]
    strand = chunks[6]
    phase = "."
    attributes = f"ID={transcript_id};Parent={gene_id}"
    string = f"{seqid}\t{source}\t{seq_type}\t{start}\t{stop}\t{score}\t{strand}\t{phase}\t{attributes};\n"
    return string

def exon_string(transcript_id, line, exon_counter):
    chunks = line.split("\t")

    seqid = chunks[0]
    source = chunks[1]
    start = chunks[3]
    stop = chunks[4]
    score = chunks[5]
    strand = chunks[6]

    seq_type = "exon"
    phase = "."
    attributes = f"ID={transcript_id}.exon{exon_counter};Parent={transcript_id}"
    exon_string = f"{seqid}\t{source}\t{seq_type}\t{start}\t{stop}\t{score}\t{strand}\t{phase}\t{attributes};\n"
    
    seq_type = "CDS"
    phase="0"
    attributes = f"ID={transcript_id}.CDS{exon_counter};Parent={transcript_id}"
    CDS_string = f"{seqid}\t{source}\t{seq_type}\t{start}\t{stop}\t{score}\t{strand}\t{phase}\t{attributes};\n"
    
    return exon_string + CDS_string

In [18]:
def write_gene(current_gene_id, gene_lines, edit, start, end):
    chunks = gene_lines[0].split("\t")
    # write the gene line first
    gene = gene_string(current_gene_id, start, end, gene_lines[0])
    edit.write(gene)
    # now write each of the transcripts
    exon_counter = 1
    for l in gene_lines:
        chunks = l.split("\t")
        if chunks[2] == "transcript":
            transcript_id = chunks[8].split(";")[1].split(" ")[2][1:-1]
            transcript = transcript_string(current_gene_id, transcript_id, l)
            edit.write(transcript)
        else:
            exon = exon_string(transcript_id, l, exon_counter)
            edit.write(exon)
            exon_counter += 1

    edit.write("###\n")

In [11]:
gene_lines = []
current_gene_id = None
start = np.inf
end = 0

with open("/Volumes/scratch/pycnogonum/transcriptome/isoseq/collapsed.gff", "r") as isoseq:
    with open("/Volumes/scratch/pycnogonum/genome/draft/annot_merge/isoseq.gff", "w") as edit:
        for line in isoseq:
            if line.startswith("#"):
                edit.write(line)
                continue
            chunks = line.split("\t")
            entry_type = chunks[2]
            # print(line)
            if entry_type == "transcript":
                gene_id = chunks[8].split(";")[0].split(" ")[1][1:-1]
                if gene_id != current_gene_id:
                    if current_gene_id is None: # first gene
                        current_gene_id = gene_id
                        gene_lines.append(line)
                        continue
                    # new gene! Write it out:
                    write_gene(current_gene_id, gene_lines, edit, start, end)
                    # reset the gene
                    current_gene_id = gene_id
                    gene_lines = []
                    start = int(chunks[3])
                    end = int(chunks[4])
                else:
                    start = min(start, int(chunks[3]))
                    end = max(end, int(chunks[4]))
            gene_lines.append(line)
        # write last gene
        write_gene(current_gene_id, gene_lines, edit, start, end)

In [19]:
gene_lines = []
current_gene_id = None

with open("/Volumes/scratch/pycnogonum/genome/draft/braker/braker/braker.gff3", "r") as braker:
    with open("/Volumes/scratch/pycnogonum/genome/draft/annot_merge/braker.gff", "w") as edit:
        for line in braker:
            chunks = line.split("\t")
            entry_type = chunks[2]

            if entry_type in ["start_codon", "stop_codon", "intron"]:
                continue

            if entry_type == "gene":
                current_gene_id = chunks[8].split(";")[0].split("=")[1]
                line = line.rstrip() + "gene_id=" + current_gene_id + ";\n"
            
            if current_gene_id in braker_unique:
                edit.write(line)

In [13]:
import matplotlib.pyplot as plt

In [14]:
def plot_overlap(df):
    # plot the overlapping regions as straight lines, color by transcript ID
    start = np.min((df["bra_start"].min(), df["iso_start"].min()))
    end = np.max((df["bra_end"].max(), df["iso_end"].max()))

    fig, ax = plt.subplots()

    # for i, transcript in df.iterrows():
    #     ax.hlines(i*2, transcript["iso_start"], transcript["iso_end"], label=transcript["iso_gene"])
    #     ax.hlines(i*2+1, transcript["bra_start"], transcript["bra_end"], label=transcript["bra_gene"])
    ax.set_xlim(start-1000, end+1000)
    ax.legend()

In [15]:
# a function that plots all the unique transcripts in a dataframe, coloring them by gene ID
def plot_overlap(df):
    # first find out the minimum start and maximum end positions of all the real BRAKER genes
    # (i.e. not the -1 values)
    bra_start_min = np.min(df["bra_start"][df["bra_start"] > 0])
    bra_end_max = np.max(df["bra_end"][df["bra_end"] > 0])
    # now do the same for the iso-seq genes
    iso_start_min = np.min(df["iso_start"][df["iso_start"] > 0])
    iso_end_max = np.max(df["iso_end"][df["iso_end"] > 0])
    # the plot should start 1000 bp before the minimum start and end 1000 bp after the maximum end
    start = np.min((bra_start_min, iso_start_min)) - 1000
    end = np.max((bra_end_max, iso_end_max)) + 1000

    fig, ax = plt.subplots()

    # get a list of all the unique gene IDs
    gene_list = np.unique(df["iso_gene"].tolist() + df["bra_gene"].tolist())
    # generate a color map for the gene IDs - this should be always less than 10, so we can take
    # the Set1 colormap
    colors = plt.cm.Set1(np.linspace(0, 1, len(gene_list)))
    # create a dictionary that maps gene IDs to colors
    cmap = dict(zip(gene_list, colors))

    # each transcript should be a straight line, colored by the gene ID. Let's do BRAKER first.
    braker = df.groupby("bra_attributes").first()
    for i, row in braker.iterrows():
        ax.hlines(i, row["bra_start"], row["bra_end"], color=cmap[row["bra_gene"]])

    # now do the same for the iso-seq genes
    iso = df.groupby("iso_attributes").first()
    for i, row in iso.iterrows():
        try:
            ax.hlines(i, row["iso_start"], row["iso_end"], color=cmap[row["iso_gene"]])
        except KeyError:
            # there might be some empty IDs here if there is no overlap for this transcript
            # ax.hlines(i, start, end, linestyles="dashed", color="black")
            continue
    
    # set the limits of the plot
    ax.set_xlim(start, end)
    # keep track of what chromosome we're in in case we want to go to the genome browser
    ax.set_title(f"{df['bra_seqname'].iloc[0]}")
    # save the plot for manual inspection
    plt.savefig(f"figs/{df['bra_gene'].iloc[0]}.png")
    # close plot to save memory
    plt.close()

In [16]:
for gene in tqdm(unknown):
    keep = bra2iso["bra_gene"] == gene
    plot_overlap(bra2iso[keep])

100%|██████████| 264/264 [00:14<00:00, 18.59it/s]


Manual inspection of the ISO-seq/BRAKER overlap for these 264 genes identifies another 49 that need
further examination. Those were aligned against uniref90 to check whether the BRAKER portions
correspond to real genes or are spurious.